In [1]:
# pip install langchain qdrant_client openai tiktoken streamlit

In [ ]:
### DO NOT RUN ALL ###

In [ ]:
### these are functions to run individually as needed to modify the Qdrant collections!! the py. files are to run the script ###

In [2]:
from dotenv import load_dotenv
import streamlit as st

In [3]:
# Imports
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from qdrant_client import QdrantClient, models
import os


In [14]:
# create qdrant client
client = QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [15]:
# create collection

# define parameters for vectors
vectors_config = models.VectorParams(
    size=1536, # vector size for OpenAI
    distance=models.Distance.COSINE
)

# create collection
client.recreate_collection(
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    vectors_config=vectors_config,
)

True

In [16]:
# create vector store

embeddings = OpenAIEmbeddings()

# get collection name
vector_store = Qdrant(
    client=client, 
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"), 
    embeddings=embeddings,
)

In [7]:
# # add 'extracted code' to vector store

# from langchain_text_splitters import Language, RecursiveCharacterTextSplitter

# def get_python_chunks(text):
    
#     python_splitter = RecursiveCharacterTextSplitter.from_language(
#     language=Language.PYTHON, chunk_size=50, chunk_overlap=0
# )
    
#     chunks = python_splitter.split_text(text)
#     return chunks

# with open('extracted_code.txt') as f:
#     raw_text = f.read()
    
# texts = get_python_chunks(raw_text)

# vector_store.add_texts(texts)

['4c14c9807837490b80841115f7f00e81',
 '87ceac3734a843e08cea02a805ac901a',
 '4e88b8f6263d4050969a2ad93ec4cec1',
 '4063d4b6468d4980afac498c5251542e',
 '1b2452e217094fbb800042d980caaa2f',
 'c1717aaa3af542678db13a0cc8b20922',
 '732a08fb58914b9ca3510ffc05f739fe',
 '50b7c70836764f27921fb9ec18ab381e',
 '7b8bf5b33282401188e75a478f0897f7',
 '2d0e2321306240ee90cd10ec45b4162a',
 'bdf18a3e9d264e44b773f7b5d059d6f7',
 'e938f8d3704b4f3a98cb3ffad809b07a',
 '1c598049905b48b48578c3d922de7c9b',
 'a44fd58257294b16845c4fe16f1c2043',
 '01bfa5879a3a42e4ae201fda5fdfad8d',
 '7fffe994c1e74f72bb1cf233c407488c',
 'ec4d34e85fc04030ac6e186ac37ec3aa',
 'f2a01c9798da41f8a959c494de294555',
 '24ce164ce62944c8b68cc85326842b76',
 '9473a21f6095463787c960c3db0c8e45',
 'd62f94aa973544ab84916e243886e6fd',
 'ef4aea52e1274353a8b013c0e0832679',
 'cc3c9ad1fd8f4a1fa010a4c159f9f444',
 '002d8d1cc99a43d1a534f2be02c99665',
 'f6fe846417aa41dba47ede0b1402ba99',
 '02a5682e2caa4e6da482de8a4e1880b2',
 'b1843a6e281b441db6586c819cfebe8a',
 

In [17]:
# add 'extracted text' to vector store

from langchain_text_splitters import CharacterTextSplitter

def get_text_chunks(text):
    
    text_splitter = CharacterTextSplitter(
        separator="\n\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
     
    chunks = text_splitter.split_text(text)
    return chunks

with open('combined_markdown_and_code.txt') as f:
    raw_text = f.read()
    
texts = get_text_chunks(raw_text)

vector_store.add_texts(texts)

Created a chunk of size 1239, which is longer than the specified 1000
Created a chunk of size 1316, which is longer than the specified 1000
Created a chunk of size 1255, which is longer than the specified 1000
Created a chunk of size 1255, which is longer than the specified 1000


Created a chunk of size 2521, which is longer than the specified 1000
Created a chunk of size 1454, which is longer than the specified 1000
Created a chunk of size 2521, which is longer than the specified 1000
Created a chunk of size 1119, which is longer than the specified 1000
Created a chunk of size 1351, which is longer than the specified 1000
Created a chunk of size 1119, which is longer than the specified 1000
Created a chunk of size 8344, which is longer than the specified 1000
Created a chunk of size 8344, which is longer than the specified 1000
Created a chunk of size 8339, which is longer than the specified 1000
Created a chunk of size 8339, which is longer than the specified 1000
Created a chunk of size 1117, which is longer than the specified 1000
Created a chunk of size 1259, which is longer than the specified 1000
Created a chunk of size 1117, which is longer than the specified 1000
Created a chunk of size 1259, which is longer than the specified 1000
Created a chunk of s

['2382d41651f648de8119cd42e3fa46b4',
 '3721114a8e3b4b938e22db66971c95d7',
 'dc048b8a76e54310a158b0d1dfd79e76',
 'c8f94e338b624b10b36b3f318f531db6',
 'dd9e5c6c3d6044dc996d520b5a2cf7af',
 '58377f5209e04e109c796821c321038b',
 '7bd17e9fec9e4c70ae07345208a7a3d1',
 '56421aed2ab94b84a7d2cb1486fd48a7',
 '54945c6c104247e091895f5afb57a058',
 '9ee8e7f171844636a8b14f3dfce6c1e3',
 'ee4562951db34684963f572553e760cb',
 '867d2f76ed994d11bb3ff9756a75229d',
 '55ca427b7d934d9f88a873934dcce747',
 '4667c52970b84bb4b0ab9a2838ad897b',
 'd59ba884e48d412d8717132897e849ef',
 'c204103daf9f47e99c0ec12c1a635e0e',
 'c40b949c966240ecbf260ee8d9bbd4f9',
 'ee655e63d6144dfab9d1672811cb4ae7',
 'caf1a9018ed64329873bd3fda33c5406',
 '49b78a375e4a4288897e985fbb392428',
 '74455bf887764f04b59669a5e58b20a2',
 'baecb40177a84169baac654e89a6e069',
 '8655000044004c84851841fb596c743d',
 '3f07c0611f4b42fe909789f424e90dfe',
 '3bff962dda15465383bc4032c84c0716',
 'af28348759db43528d52d850647abc65',
 '55f704876e5b448a96cb3ab4c329662b',
 

In [13]:
# delete collection

client.delete_collection(collection_name=os.getenv("QDRANT_COLLECTION_NAME"))

True

In [18]:
from langchain.schema import retriever

from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA

# add vector store to retrieval chain

retrievalQA = RetrievalQA.from_llm(
    llm=OpenAI(),
    retriever=vector_store.as_retriever()
    )

In [10]:
# # conversational retrieval qa
# from langchain.chains import ConversationalRetrievalChain
# from langchain.chat_models import ChatOpenAI

# llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)
# crc = ConversationalRetrievalChain.from_llm(llm,retriever)

In [19]:
query = "What topic are you knowledgable about?"

response = retrievalQA.run(query)

print(response)

 I am knowledgeable about machine learning and natural language processing.
